# 3. Classifier Prediction on Twitter Data

Daniel Ruiz, MSc in Data Science and Business Analytics (DSBA), Bocconi University

Reference codes (alphabetically):

## 3.1. Loading packages and creating folder

In [1]:
# general
from BOW import *
import pandas as pd
import numpy as np
import pickle as pkl

## 3.2. Prediction on All Tweets

In [19]:
my_companies = ['br_embraer',
                'br_americanas',
                'br_pontofrio',
                'br_petrobras',
                'br_bradesco',
                'br_renner',
                'br_gol',
                'br_magazineluiza',
                'br_itau',
                'br_valor',
                'us_abercrombie',
                'us_boeing',
                'us_beyondmeat',
                'us_morganstanley',
                'us_jpmorgan',
                'us_exxonmobil',
                'us_americanair',
                'us_cocacola',
                'us_tesla',
                'us_wsj']

for company in my_companies:

    # load and filter data
    print(company)
    df = pd.read_pickle('Dataset_Twitter_Clean_03/'+company+'.pkl')
    df = df[['snowball_stems','datetime','has_happy','has_sad']]
    print('Test set size:',df.shape[0])

    # classify -> noisy labels
    df['noisy_pos'] = df.has_happy>df.has_sad

    # classify -> classifier (logistic with info words)
    if company[:2]=='us':
        folder_clas='Classifiers/English/'
    else:
        folder_clas='Classifiers/Portuguese/'
        
    my_classifier = pkl.load(open(folder_clas+'skc_logistic_inf.sav', 'rb'))
    df['stem_bow']=df.apply(lambda x: bag_of_words(x.snowball_stems), axis=1)
    df['clas_pos']=df.apply(lambda x: my_classifier.classify(x.stem_bow), axis=1)

    # select classification
    df['use_classifier'] = df.has_happy==df.has_sad
    df['final_pos'] = np.where(df.use_classifier==1,df.clas_pos,df.noisy_pos)

    # reduce dataframe
    df = df[['datetime','final_pos']]

    #save
    df.to_pickle('Dataset_TS_01/'+company+'.pkl')

br_embraer
Test set size: 1407
